In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import cv2

np.set_printoptions(linewidth=np.inf)
np.set_printoptions(edgeitems=30) 

In [ ]:


img = cv2.imread('data/pets/test/cat/9999.jpg', cv2.IMREAD_GRAYSCALE)

print(img)

plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title("Exemple d'image")
plt.show()

# Inspection des métadonnées de l'image
print(f"Shape (dimensions)    : {img.shape}")
print(f"Type des valeurs      : {img.dtype}")